In [121]:
import pygrib
import pandas as pd
import xarray as xr
import datetime as dt
import rioxarray as rio
import numpy as np

In [154]:
file_to_open_nc = '/home/daniele/documents/github/ftt01/phd/data/meteo/providers/dwd/example/icon-d2-eps_germany_icosahedral_single-level_2022070800_001_2d_tot_prec.grib2'
file_to_open_nc = '/home/daniele/documents/github/ftt01/phd/data/meteo/providers/dwd/example/t.grib2'
ncf = pygrib.open(file_to_open_nc)

In [155]:
ncf.messagenumber

0

In [136]:
a = ncf[1]

In [146]:
a.data()

ValueError: unsupported grid unstructured_grid

In [3]:
lat_min = 45.00
lat_max = 47.50
lon_min = 9.80
lon_max = 13.00
basin = 'trentino_alto_adige'

In [88]:
tt = np.array([[[0 for k in range(1,126)] for j in range(1,161)] for i in range(1,21)])
for m in range(1,ncf.messages):
    data, lats, lons = ncf[m].data(lat1=lat_min,lat2=lat_max,lon1=lon_min,lon2=lon_max)
    data_eps.append(data.data)

In [92]:
t = data.data

In [94]:
t.shape

(125, 160)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [103]:
np.concatenate( t, np.array( [i for i in range(1,21)] ))

TypeError: only integer scalar arrays can be converted to a scalar index

In [82]:
# put data into a dataset
ds = xr.Dataset(
    data_vars=dict(
        var=(["lat","lon","ens"], data.data)
    ),
    coords=dict(
        lat=( ["lat"], np.array([l[0] for l in lats]) ),
        lon=( ["lon"], lons[0] ),
        ens=( ["ens"], range(1,21) )
    ),
    attrs=dict(
        creation_date=str(dt.datetime.now()),
        author='ftt01 (dallatorre.daniele@gmail.com)',
        copyright='DWD'
        ),
)

/home/daniele/.local/lib/python3.10/site-packages/xarray/core/variable.py:231: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(data)


ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['lat', 'lon', 'ens'], [array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.88281250e-04, 2.44140625e-03, 2.44140625e-03, ...,
        1.44189453e+00, 2.26806641e+00, 2.21777344e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.76660156e+00, 2.64404297e+00, 2.64404297e+00],
       [0.00000000e+00, 9.76562500e-04, 9.76562500e-04, ...,
        1.97558594e+00, 1.97558594e+00, 2.65869141e+00]]), 0]) to Variable.

In [80]:
ds

<xarray.Dataset>
Dimensions:  (lat: 125, lon: 160, ens: 20)
Coordinates:
  * lat      (lat) float64 45.0 45.02 45.04 45.06 ... 47.42 47.44 47.46 47.48
  * lon      (lon) float64 9.82 9.84 9.86 9.88 9.9 ... 12.94 12.96 12.98 13.0
  * ens      (ens) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Data variables:
    var      (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 1.816 1.976 1.976 2.659
Attributes:
    creation_date:  2022-07-08 18:52:38.621024
    author:         ftt01 (dallatorre.daniele@gmail.com)
    copyright:      DWD

In [72]:
ds = ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)

In [73]:
ds = ds.rio.write_crs("EPSG:4326", inplace=True)

In [74]:
ds.rio.to_raster('/home/daniele/documents/github/ftt01/phd/data/meteo/providers/dwd/example/remapped.tif')

In [77]:
ds

<xarray.Dataset>
Dimensions:  (lat: 125, lon: 160, ens: 20)
Coordinates:
  * lat      (lat) float64 45.0 45.02 45.04 45.06 ... 47.42 47.44 47.46 47.48
  * lon      (lon) float64 9.82 9.84 9.86 9.88 9.9 ... 12.94 12.96 12.98 13.0
  * ens      (ens) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Data variables:
    ens0     (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 1.816 1.976 1.976 2.659
Attributes:
    creation_date:  2022-07-08 18:52:19.213525
    author:         ftt01 (dallatorre.daniele@gmail.com)
    copyright:      DWD